<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting/blob/main/prepareTransactions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

Run date time (IST): 2024-06-30 22:14:37


In [2]:
!pip install yfinance
!pip install numpy_financial
!pip install pyxirr

In [3]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

myts_5ps = pd.read_csv('/content/drive/My Drive/data/stocks/myTransactions_5Paisa.csv')
myts_zrd = pd.read_csv('/content/drive/My Drive/data/stocks/myTransactions_Zerodha.csv')
mypfs = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/myPortfolioStocks.csv')
mypfs = mypfs[mypfs['InPortfolio'].isin(['SV','DM'])]['Symbol']

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
myts_zrd['amount'] = myts_zrd['quantity'] * myts_zrd['price']
#tmp_df = myts_zrd[myts_zrd['symbol'].isin(['SBIN', 'AAVAS'])]

In [5]:

'''
myts_zrd.groupby(['order_id', 'trade_date', 'symbol', 'trade_type']).apply(lambda x: pd.Series({
    'count': x['quantity'].count(),
    'price': round((x['price'] * x['quantity']).sum() / x['quantity'].count(),2)
})).reset_index()
'''
myts_zrd_grp = myts_zrd.groupby(['trade_date', 'symbol', 'trade_type'])[['quantity', 'amount']].sum().reset_index()
#myts_zrd_grp
myts_zrd_grp

,trade_date,symbol,trade_type,quantity,amount
0,02-05-2024,AVANTIFEED,sell,186,104022.00
1,04-03-2024,SBIETFPB,buy,107,25622.22
2,04-09-2023,BANDHANBNK,buy,127,30041.85
3,04-09-2023,JPPOWER,buy,3620,30227.00
4,04-10-2023,BRITANNIA,buy,13,58383.00
...,...,...,...,...,...
158,31-08-2023,ARE&M,buy,48,30048.00
159,31-08-2023,CAMS,buy,13,30829.85
160,31-08-2023,LUXIND,buy,21,30792.65
161,31-08-2023,METROPOLIS,buy,23,30995.10


In [6]:
myts_zrd_grp['trade_date'] = pd.to_datetime(myts_zrd_grp['trade_date'], format='%d-%m-%Y').dt.strftime('%Y-%m-%d')

In [7]:
myts_5ps = myts_5ps.rename(columns={'Transaction Date': 'trade_date', 'Symbol': 'symbol', 'Type': 'trade_type', 'Quantity': 'quantity', 'Price': 'price'})
columns = ['trade_date', 'symbol', 'trade_type', 'quantity', 'price']
myts_5ps = myts_5ps[columns]

In [8]:
myts_5ps['amount'] = myts_5ps['quantity'] * myts_5ps['price']
myts_5ps = myts_5ps.groupby(['trade_date', 'symbol', 'trade_type'])[['quantity', 'amount']].sum().reset_index()
myts_5ps.loc[myts_5ps['trade_type'] == 'Buy', 'trade_type'] = 'buy'
myts_5ps.loc[myts_5ps['trade_type'] == 'Sell', 'trade_type'] = 'sell'
myts_5ps

,trade_date,symbol,trade_type,quantity,amount
0,Apr 03 2023,JUNIORBEES,buy,83,33302.92
1,Apr 03 2023,NIFTYBEES,buy,174,33143.52
2,Apr 03 2023,RELETFNM150,buy,285,33077.10
3,Apr 03 2024,ICICIGI,sell,56,95259.36
4,Apr 03 2024,RBLBANK,buy,310,77407.00
...,...,...,...,...,...
300,Sep 01 2023,BAJAJHIND,buy,1200,30060.00
301,Sep 01 2023,GLAND,buy,17,30005.00
302,Sep 01 2023,GLAND,sell,55,95101.39
303,Sep 04 2023,GLAND,buy,17,29187.13


In [9]:
myts_5ps['trade_date'] = pd.to_datetime(myts_5ps['trade_date'], format='%b %d %Y').dt.strftime('%Y-%m-%d')
#myts_5ps = myts_5ps[myts_5ps['symbol'].isin(['SBIN', 'AAVAS'])]

In [10]:
df_myts = pd.concat([myts_zrd_grp, myts_5ps], ignore_index=True)
df_myts.loc[df_myts['trade_type'] == 'buy', 'amount'] *= -1
df_myts.loc[df_myts['trade_type'] == 'sell', 'quantity'] *= -1

In [11]:
tmp_df = df_myts.groupby(['symbol'])[['quantity']].sum().reset_index()
tmp_df = tmp_df[tmp_df['quantity'] > 0]
tmp_df

,symbol,quantity
0,5PAISA,331
1,AARTIIND,167
2,AAVAS,48
7,ASIANPAINT,68
8,ASIANTILES,1244
...,...,...
113,UJJIVANSFB,1700
114,UTIAMC,158
115,VAIBHAVGBL,214
116,VALIANTORG,260


In [12]:
import yfinance as yfin
yfin.pdr_override()
import pandas as pd
from datetime import date, timedelta

def get_curr_trade(stock_symbol, quantity):

  stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='1d', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
  curr_date = (date.today() + timedelta(days=1)).strftime('%Y-%m-%d')
  price = round(stock_df['Close'].values[0],2)
  amount = round(quantity * price, 2)

  return   pd.DataFrame.from_dict({'trade_date': [curr_date],
                          'symbol': [stock_symbol],
                          'trade_type':['sell'],
                          'quantity': [quantity],
                          'amount': [amount]})

for index, row in tmp_df.iterrows():
  stock_symbol = row['symbol']
  quantity = row['quantity']
  tmp_df = get_curr_trade(stock_symbol, quantity)
  df_myts = pd.concat([df_myts, tmp_df], ignore_index=True)

  #print(get_curr_trade(stock_symbol, quantity))

yfinance: pandas_datareader support is deprecated & semi-broken so will be removed in a future verison. Just use yfinance.


In [13]:
import pandas as pd
import numpy as np
import numpy_financial as npf
from pyxirr import xirr

# Sample DataFrame
'''data = {
    'trade_date': ['2022-01-01', '2022-04-01', '2023-01-01', '2023-04-01'],
    'symbol': ['AAPL', 'AAPL', 'GOOG', 'GOOG'],
    'trade_type': ['buy', 'sell', 'buy', 'sell'],
    'amount': [-1000, 1200, -2000, 2200]
}
'''
df = df_myts

# Define function to calculate XIRR
def calculate_xirr(cash_flows):
    #print(cash_flows)
    dates = pd.to_datetime(cash_flows['trade_date'])
    amounts = cash_flows['amount']

    # Check if both negative and positive amounts exist
    if (amounts < 0).any() and (amounts > 0).any():
        return xirr(pd.DataFrame({"dates": dates, "amounts": amounts}))
    else:
        return np.nan  # Return NaN if not both types of cash flows are present

# Define function to calculate XIRR
def calculate_abs(cash_flows):
    #print(cash_flows)
    dates = pd.to_datetime(cash_flows['trade_date'])
    amounts = cash_flows['amount']

    # Check if both negative and positive amounts exist
    invested = round(-sum(cash_flows[cash_flows['trade_type'] == 'buy']['amount']),0)
    compounded = round(sum(cash_flows[cash_flows['trade_type'] == 'sell']['amount']),0)

    if invested == 0:
      return np.nan  # Or any other appropriate value to indicate this situation
    else:
      abs = (compounded - invested)/invested

      return abs

# Group by 'symbol' and calculate XIRR and CAGR
df_xirr = df.groupby('symbol').apply(lambda x: pd.Series({
    'XIRR%': round(calculate_xirr(x)*100,0),
     'ABS%': round(calculate_abs(x)*100,0)
})).reset_index()

# Print results
print('-----------')
print('CAGR%: ', round(calculate_xirr(df_myts)*100,2))
print('-----------')

-----------
CAGR%:  27.92
-----------


In [15]:
df_xirr['InPF'] = 0
df_xirr.loc[df_xirr['symbol'].isin(mypfs), 'InPF'] = 1
df_xirr.sort_values(by = 'XIRR%', ascending=False)

,symbol,XIRR%,ABS%,InPF
25,CASTROLIND,2824.0,30.0,0
48,HINDPETRO,1817.0,34.0,0
89,REPCOHOME,976.0,3.0,1
64,JSWSTEEL,802.0,29.0,0
60,J&KBANK,680.0,35.0,0
...,...,...,...,...
84,RAJESHEXPO,-50.0,-46.0,1
99,SPORTKING,-68.0,-6.0,0
22,BOROLTD,-75.0,-4.0,1
74,MASFIN,-93.0,-70.0,1


In [16]:
df_xirr = df_xirr.rename(columns={'symbol': 'Symbol'})
df_xirr.to_csv('/content/drive/My Drive/data/stocks/myStocks-XIRR.csv', index=False)

In [17]:
symbol = 'CAMS'
df_myts[df_myts['symbol'] == symbol]

,trade_date,symbol,trade_type,quantity,amount
149,2023-11-28,CAMS,buy,34,-93738.00
159,2023-08-31,CAMS,buy,13,-30829.85
250,2023-07-07,CAMS,buy,13,-30222.79
454,2023-10-19,CAMS,buy,12,-30377.64
485,2024-07-01,CAMS,sell,72,260118.00
